In [1]:
#importing basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("urldata.csv")
data = data.drop(['Domain'], axis = 1).copy()
# data = pd.read_csv('Training Dataset.arff', header=None, comment='@')

In [3]:
data.isnull().sum()

Have_IP          0
Have_At          0
URL_Length       0
URL_Depth        0
Redirection      0
https_Domain     0
TinyURL          0
Prefix/Suffix    0
DNS_Record       0
Web_Traffic      0
Domain_Age       0
Domain_End       0
iFrame           0
Mouse_Over       0
Right_Click      0
Web_Forwards     0
Google_Index     0
count_dot        0
count_www        0
count_per        0
count_ques       0
count_hyphen     0
count_equal      0
Label            0
dtype: int64

In [4]:
# shuffling the rows in the dataset so that when splitting the train and test set are equally distributed
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,...,Right_Click,Web_Forwards,Google_Index,count_dot,count_www,count_per,count_ques,count_hyphen,count_equal,Label
0,0,0,1,4,0,0,0,0,0,1,...,1,1,1,1,0,0,0,5,0,0
1,0,0,1,3,0,0,0,0,0,0,...,1,0,1,6,0,0,1,0,1,1
2,0,0,1,6,0,0,0,0,0,1,...,1,0,1,1,0,0,0,2,0,0
3,0,0,0,2,0,0,0,0,0,1,...,1,1,1,1,0,0,0,0,0,1
4,0,0,1,0,0,0,0,1,0,0,...,1,0,1,3,0,0,0,1,0,1


In [5]:
# Sepratating & assigning features and target columns to X & y
y = data['Label']
X = data.drop('Label',axis=1)
# X = data.iloc[:, :-1]
# y = data.iloc[:, -1]
X.shape, y.shape

column_name=list(X.columns)

In [6]:
# Splitting the dataset into train and test sets: 80-20 split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2, random_state = 12)
X_train.shape, X_test.shape

((8000, 23), (2000, 23))

In [7]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

rf=RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

y_train_rf = rf.predict(X_train)
y_test_rf =rf.predict(X_test)


#computing the accuracy of the model performance
acc_train_rf= accuracy_score(y_train,y_train_rf)
acc_test_rf = accuracy_score(y_test,y_test_rf)

print("Random Forest: Accuracy on training Data: {:.3f}".format(acc_train_rf*100))
print("Random Forest: Accuracy on test Data: {:.3f}".format(acc_test_rf*100))

print(classification_report(y_test, y_test_rf))

Random Forest: Accuracy on training Data: 99.650
Random Forest: Accuracy on test Data: 98.750
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       982
           1       0.99      0.98      0.99      1018

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



In [8]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=20, random_state=0, shuffle=True)
results = cross_val_score(rf, X_train, y_train, cv=kfold)
print("K-Fold Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

K-Fold Accuracy: 98.312% (0.467%)


In [9]:
from sklearn.feature_selection import RFECV

## create ranking among all features by selecting only one
rfecv = RFECV(RandomForestClassifier(random_state=0), step=1, cv=10, scoring='accuracy')
rfecv.fit(X_train, y_train)

for i in range(X.shape[1]):
    print(column_name[i],': Selected %s, Rank: %.3f' % (rfecv.support_[i], rfecv.ranking_[i]))


Have_IP : Selected False, Rank: 2.000
Have_At : Selected True, Rank: 1.000
URL_Length : Selected True, Rank: 1.000
URL_Depth : Selected True, Rank: 1.000
Redirection : Selected True, Rank: 1.000
https_Domain : Selected True, Rank: 1.000
TinyURL : Selected True, Rank: 1.000
Prefix/Suffix : Selected True, Rank: 1.000
DNS_Record : Selected True, Rank: 1.000
Web_Traffic : Selected True, Rank: 1.000
Domain_Age : Selected True, Rank: 1.000
Domain_End : Selected True, Rank: 1.000
iFrame : Selected True, Rank: 1.000
Mouse_Over : Selected True, Rank: 1.000
Right_Click : Selected True, Rank: 1.000
Web_Forwards : Selected True, Rank: 1.000
Google_Index : Selected True, Rank: 1.000
count_dot : Selected True, Rank: 1.000
count_www : Selected True, Rank: 1.000
count_per : Selected True, Rank: 1.000
count_ques : Selected True, Rank: 1.000
count_hyphen : Selected True, Rank: 1.000
count_equal : Selected True, Rank: 1.000


In [10]:
X_train.columns[rfecv.support_]

Index(['Have_At', 'URL_Length', 'URL_Depth', 'Redirection', 'https_Domain',
       'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic', 'Domain_Age',
       'Domain_End', 'iFrame', 'Mouse_Over', 'Right_Click', 'Web_Forwards',
       'Google_Index', 'count_dot', 'count_www', 'count_per', 'count_ques',
       'count_hyphen', 'count_equal'],
      dtype='object')

In [11]:
print('Optimal number of features: {}'.format(rfecv.n_features_))

Optimal number of features: 22


In [12]:
feature_importance = list(zip(column_name, rfecv.support_))
new_features = []
for key,value in enumerate(feature_importance):
    if(value[1]) == True:
        new_features.append(value[0])
        
print(new_features)

['Have_At', 'URL_Length', 'URL_Depth', 'Redirection', 'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic', 'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over', 'Right_Click', 'Web_Forwards', 'Google_Index', 'count_dot', 'count_www', 'count_per', 'count_ques', 'count_hyphen', 'count_equal']


In [13]:
X_new = data[new_features]
X_new.head()

X_train_new, X_test_new, y_train, y_test = train_test_split(X_new, y, 
                                                    test_size = 0.2, random_state = 12)

In [14]:
rf = RandomForestClassifier(random_state=0)

rf.fit(X_train_new,y_train)

y_train_rf = rf.predict(X_train_new)
y_test_rf =rf.predict(X_test_new)


#computing the accuracy of the model performance
acc_train_rf= accuracy_score(y_train,y_train_rf)
acc_test_rf = accuracy_score(y_test,y_test_rf)

# print("Random Forest: Accuracy on training Data: {:.3f}".format(acc_train_rf*100))
print("Feature Selection Accuracy: {:.3f}".format(acc_test_rf*100))

print(classification_report(y_test, y_test_rf))

Feature Selection Accuracy: 98.750
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       982
           1       0.99      0.98      0.99      1018

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



In [15]:
from sklearn.model_selection import GridSearchCV, cross_val_score
param_grid = {
    'bootstrap': [True],
    'max_depth': [10,15,20],
    'max_features': ['auto', 'sqrt', 'log2'],    
    'criterion' :['gini', 'entropy']
    }
grid = GridSearchCV(estimator = rf, param_grid = param_grid , scoring = 'accuracy', verbose = 1, n_jobs = -1, cv=20)
grid.fit(X_train_new,y_train)
print("Best Score:" + str(grid.best_score_))
print("Best Parameters: " + str(grid.best_params_))
best_parameters = grid.best_params_

Fitting 20 folds for each of 18 candidates, totalling 360 fits
Best Score:0.9840000000000002
Best Parameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': 20, 'max_features': 'auto'}


In [16]:
print(best_parameters)

{'bootstrap': True, 'criterion': 'gini', 'max_depth': 20, 'max_features': 'auto'}


In [24]:
rf = RandomForestClassifier(**best_parameters, random_state=0)

rf.fit(X_train_new,y_train)

y_train_rf = rf.predict(X_train_new)
y_test_rf =rf.predict(X_test_new)


#computing the accuracy of the model performance
acc_train_rf= accuracy_score(y_train,y_train_rf)
acc_test_rf = accuracy_score(y_test,y_test_rf)

# print("Random Forest: Accuracy on training Data: {:.3f}".format(acc_train_rf))
print("Hyper Parameter Tuning Accuracy: {:.3f}".format(acc_test_rf*100))

print(classification_report(y_test, y_test_rf))

Hyper Parameter Tuning Accuracy: 98.800
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       982
           1       0.99      0.98      0.99      1018

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



In [25]:
X_test_new

,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,...,Mouse_Over,Right_Click,Web_Forwards,Google_Index,count_dot,count_www,count_per,count_ques,count_hyphen,count_equal
5669,0,1,2,0,0,1,0,0,1,1,...,0,1,0,1,1,0,0,0,11,0
8800,0,1,10,0,0,0,0,0,1,1,...,0,1,0,1,1,0,0,0,0,0
3205,1,1,1,0,0,0,0,0,1,1,...,0,1,0,1,6,1,2,0,9,1
8731,0,1,4,0,0,0,0,0,1,1,...,1,1,1,1,1,0,0,0,0,0
6412,0,1,1,0,0,0,0,1,1,1,...,1,1,1,1,3,0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,0,1,3,0,0,0,0,0,1,0,...,1,1,1,1,1,0,0,0,9,0
142,0,1,3,0,0,0,0,0,1,1,...,0,1,0,1,1,0,0,0,9,0
275,0,1,3,0,0,0,0,0,1,0,...,1,1,1,1,4,0,0,0,0,0
8265,0,0,1,0,0,0,1,1,1,1,...,1,1,1,1,4,0,0,0,1,0


In [26]:
y_test

5669    0
8800    0
3205    1
8731    1
6412    1
       ..
794     0
142     0
275     0
8265    1
7950    1
Name: Label, Length: 2000, dtype: int64

In [27]:
import pickle

with open('rf.pkl', 'wb') as file:
    pickle.dump(rf, file)

In [28]:
rf

RandomForestClassifier(max_depth=20, max_features='auto', random_state=0)

In [29]:
from sklearn.metrics import roc_auc_score
rf_y_pred = rf.predict_proba(X_test_new)[:, 1] # predicted probabilities

# Calculate the ROC AUC score
roc_auc = roc_auc_score(y_test, rf_y_pred)
print("ROC AUC:", roc_auc)

ROC AUC: 0.9989041449429614
